In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import ast
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, precision_recall_curve, recall_score, f1_score
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression

In [2]:
os.getcwd()

'C:\\Users\\brian\\Documents\\Irvine MSBA 2022\\Spring 2022\\Natural Language Processing\\group_project'

In [3]:
animes_df = pd.read_csv(r"C:\Users\brian\Documents\Irvine MSBA 2022\Spring 2022\Natural Language Processing\group_project\data\animes.csv")
# profiles_df = pd.read_csv(r"C:\Users\brian\Documents\Irvine MSBA 2022\Spring 2022\Natural Language Processing\group_project\data\profiles.csv")
# reviews_df = pd.read_csv(r"C:\Users\brian\Documents\Irvine MSBA 2022\Spring 2022\Natural Language Processing\group_project\data\reviews.csv")
ratings_df = pd.read_csv(r"C:\Users\brian\Documents\Irvine MSBA 2022\Spring 2022\Natural Language Processing\group_project\data\AnimeList.csv")

In [5]:
animes_df.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


In [6]:
ratings_df = ratings_df[['anime_id','rating']].copy()

In [7]:
ratings_df.anime_id.nunique()

14478

In [8]:
animes_df.uid.nunique()

16216

In [9]:
ratings_df.rename(columns={'anime_id':'uid'},inplace=True)

In [10]:
animes_df = pd.concat([animes_df, ratings_df], axis=1, join="inner")

In [11]:
animes_df.isna().sum()/len(animes_df)

uid           0.000000
title         0.000000
synopsis      0.050559
genre         0.000000
aired         0.000000
episodes      0.030460
members       0.000000
popularity    0.000000
ranked        0.114588
score         0.020238
img_url       0.011051
link          0.000000
uid           0.000000
rating        0.000000
dtype: float64

In [12]:
animes_df.shape

(14478, 14)

In [13]:
animes_df.drop_duplicates(inplace=True)

In [14]:
animes_df.shape

(14478, 14)

In [15]:
animes_df2 = animes_df[['title', 'synopsis', 'genre', 'rating']].copy()

In [16]:
animes_df2.dropna(inplace=True)

In [17]:
animes_df2.rating.value_counts()

PG-13 - Teens 13 or older         4768
G - All Ages                      4323
PG - Children                     1216
Rx - Hentai                       1143
R - 17+ (violence & profanity)     940
R+ - Mild Nudity                   840
None                               516
Name: rating, dtype: int64

In [18]:
def extract_genre(genre_list):
    return ast.literal_eval(genre_list)

In [19]:
anime_word_dict = {}
for i in animes_df2.genre:
    x = extract_genre(i)
    if x == None:
        pass
    else:
        for j in x:
            if j in anime_word_dict:
                anime_word_dict[j] += 1
            else:
                anime_word_dict[j] = 1
    

In [20]:
anime_word_dict

{'Comedy': 4705,
 'Sports': 589,
 'Drama': 2191,
 'School': 1367,
 'Shounen': 1663,
 'Music': 1698,
 'Romance': 1520,
 'Sci-Fi': 1985,
 'Adventure': 2255,
 'Mystery': 608,
 'Fantasy': 2376,
 'Action': 2947,
 'Military': 480,
 'Magic': 832,
 'Supernatural': 1212,
 'Vampire': 110,
 'Slice of Life': 1538,
 'Demons': 417,
 'Historical': 897,
 'Super Power': 495,
 'Mecha': 833,
 'Parody': 540,
 'Samurai': 171,
 'Seinen': 687,
 'Police': 217,
 'Psychological': 300,
 'Josei': 86,
 'Space': 404,
 'Kids': 2027,
 'Shoujo Ai': 64,
 'Ecchi': 574,
 'Shoujo': 558,
 'Horror': 358,
 'Shounen Ai': 88,
 'Cars': 111,
 'Martial Arts': 290,
 'Game': 281,
 'Thriller': 121,
 'Dementia': 379,
 'Harem': 278,
 'Hentai': 1208,
 'Yaoi': 42,
 'Yuri': 41}

In [21]:
adult_genres = ['Hentai', 'Yaoi', 'Yuri', 'Shounen Ai', 'Shoujo Ai', 'Ecchi']

In [22]:
adult_ratings = ['PG-13 - Teens 13 or older', 'Rx - Hentai', 'R - 17+ (violence & profanity)', 'R+ - Mild Nudity']

In [23]:
def genre_classify(text):
    genres = ast.literal_eval(text)
    adults = []
    kids = []
    for i in genres:
        if i in adult_genres:
            adults.append(i)
        else:
            kids.append(i)
    #print(adults)
    if len(adults)>0:
        classify = 1
    else:
        classify = 0
    return classify

In [24]:
animes_df2['classify'] = animes_df2.genre.apply(genre_classify)

In [25]:
def rating_classify(text):
    adults = []
    kids = []
    if text in adult_ratings:
        adults.append(text)
    else:
        kids.append(text)
    #print(adults)
    if len(adults)>0:
        classify = 1
    else:
        classify = 0
    return classify

In [26]:
animes_df2['classify2'] = animes_df2.rating.apply(rating_classify)

In [27]:
animes_df2['adult'] = animes_df2['classify'] + animes_df2['classify2']

In [28]:
animes_df2['adult'] = np.where(animes_df2['adult'] > 0, 1, 0)

In [29]:
animes_df2.drop(columns=['classify', 'classify2'],inplace=True)

In [30]:
animes_df2.head()

,title,synopsis,genre,rating,adult
0,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",PG-13 - Teens 13 or older,1
1,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...",PG-13 - Teens 13 or older,1
2,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...",PG - Children,0
3,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...",PG-13 - Teens 13 or older,1
4,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']",PG-13 - Teens 13 or older,1


In [31]:
def text_cleaner(text):
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(string = text, pattern = r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", repl = " ")
    text = re.sub(string = text, pattern = r'\(Source [A-Za-z0-9\w\W]+\)', repl = "")   # this is a big one we need to change/get rid of
    text = re.sub(string = text, pattern = r'\!|\?|\.|\,|\:|\;|\"|\'|\*|\[|\]|\(|\)|\/', repl = " ")
    text = re.sub(string = text, pattern = r' +', repl = " ")
    text = text.lower()
    text = ' '.join(text.split())
    return text

In [32]:
for x in animes_df2['synopsis']:
    text = text_cleaner(str(x))    
    words = word_tokenize(text)  

In [33]:
vectorizer = CountVectorizer()

In [34]:
vectorizer.fit(words)

CountVectorizer()

In [35]:
animes_df2.reset_index(inplace=True)

In [36]:
animes_df2.drop(columns=['index'],inplace=True)

In [37]:
animes_df2.to_csv(r"C:\Users\brian\Documents\Irvine MSBA 2022\Spring 2022\Natural Language Processing\group_project\animes_df.csv")

In [38]:
animes_df2.shape

(13746, 5)

In [41]:
dic = {}

counter = 0
while counter < len(animes_df2):
    name = str(animes_df2['title'][counter])
    one_col = word_tokenize(animes_df2['synopsis'][counter])
    filtered = [w for w in one_col if not w.lower() in stopwords.words('english')]
    dic[str(name)] = filtered
    counter += 1

In [42]:
dic

{'Haikyuu!! Second Season': ['Following',
  'participation',
  'Inter-High',
  ',',
  'Karasuno',
  'High',
  'School',
  'volleyball',
  'team',
  'attempts',
  'refocus',
  'efforts',
  ',',
  'aiming',
  'conquer',
  'Spring',
  'tournament',
  'instead',
  '.',
  'receive',
  'invitation',
  'long-standing',
  'rival',
  'Nekoma',
  'High',
  ',',
  'Karasuno',
  'agrees',
  'take',
  'part',
  'large',
  'training',
  'camp',
  'alongside',
  'many',
  'notable',
  'volleyball',
  'teams',
  'Tokyo',
  'even',
  'national',
  'level',
  'players',
  '.',
  'playing',
  'toughest',
  'teams',
  'Japan',
  ',',
  'hope',
  'sharpen',
  'skills',
  ',',
  'also',
  'come',
  'new',
  'attacks',
  'would',
  'strengthen',
  '.',
  'Moreover',
  ',',
  'Hinata',
  'Kageyama',
  'attempt',
  'devise',
  'powerful',
  'weapon',
  ',',
  'one',
  'could',
  'possibly',
  'break',
  'sturdiest',
  'blocks',
  '.',
  'Facing',
  'may',
  'last',
  'chance',
  'victory',
  'senior',
  'playe

In [43]:
animes_df2.genre = animes_df2.genre.apply(lambda s: list(ast.literal_eval(s)))

In [44]:
genre_series = pd.Series(animes_df2.genre)

In [45]:
pd.get_dummies(genre_series.apply(pd.Series).stack()).sum(level=0)

C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  """Entry point for launching an IPython kernel.


,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13741,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13742,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
13743,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
13744,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
animes_df2.head()

,title,synopsis,genre,rating,adult
0,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"[Comedy, Sports, Drama, School, Shounen]",PG-13 - Teens 13 or older,1
1,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"[Drama, Music, Romance, School, Shounen]",PG-13 - Teens 13 or older,1
2,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"[Sci-Fi, Adventure, Mystery, Drama, Fantasy]",PG - Children,0
3,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","[Action, Military, Adventure, Comedy, Drama, M...",PG-13 - Teens 13 or older,1
4,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"[Action, Mystery, Supernatural, Vampire]",PG-13 - Teens 13 or older,1


In [47]:
print("adults =",len(animes_df2[animes_df2.adult==1]))
    
print("kids =",len(animes_df2[animes_df2.adult==0]))

adults = 8531
kids = 5215


In [ ]:
animes_df3 = pd.read_csv(r"C:\Users\brian\Documents\Irvine MSBA 2022\Spring 2022\Natural Language Processing\group_project\animes_df3.csv")

In [56]:
animes_df2.dropna(inplace=True)
animes_df2.isna().sum().sum()

0

In [57]:
X = animes_df2.synopsis
y = animes_df2.adult

In [58]:
x_train, x_test, y_train, y_test = train_test_split(
            X, y, test_size = 0.3, random_state=42)

In [59]:
cv = CountVectorizer()

X= cv.fit_transform(x_train)

In [60]:
train_array = X.toarray()
train_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [178]:
tfidf = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)

In [179]:
x_tfidf = tfidf.fit_transform(x_train)
x_tfidf

<9616x18503 sparse matrix of type '<class 'numpy.float64'>'
	with 249229 stored elements in Compressed Sparse Row format>

In [180]:
x_tfidf_test = tfidf.fit_transform(x_test)

In [181]:
x_train_array = x_tfidf.toarray()

In [188]:
x_train_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [65]:
x_train

10349    episode features part story detective tries hu...
11768    pickles made fermenting vegetables rice bran s...
12203    lancer group emperor defeated every racing tea...
1612     anime centers four girls full bloom youth work...
939      fear survival instinct thrown foreign land not...
                               ...                        
5193     future humans living colonized planets control...
13423    music video song collaborative group formed fe...
5392     companions continue offer service revenge wron...
860      top-class martial artist prodigy anything-goes...
7272     years political tension mistrust two rival nat...
Name: synopsis, Length: 9616, dtype: object

In [66]:
vectorizer = CountVectorizer(max_features=2000, max_df=0.7, min_df=20)
vectorizer.fit(x_train)
x_train.vecs = vectorizer.transform(x_train)
x_test.vecs = vectorizer.transform(x_test)

In [67]:
def evaluate(preds, labels, print_details=True):
  f1 = f1_score(labels, preds, average='micro')
  if print_details:
    print('Precision:', precision_score(labels, preds, average=None))  #Computes precision
    print('Recall:', recall_score(labels, preds, average=None))        #Computes recall
    print('F1 score:', f1_score(labels, preds, average=None))          #Computes f1_score
  print('Avg. F1 score:', f1)

def training(model, train, class_size, print_details=True):
  model.fit(x_train.vecs, y_train)
  train_preds = model.predict(x_train.vecs)
  if print_details:
    dev_probs = model.predict_proba(x_train.vecs)
    plot_pr_curves(dev_probs, y_train, class_size)
  evaluate(train_preds, y_train, print_details)
  

def testing(model, test, class_size, print_details=True):
  test_preds = model.predict(x_test.vecs)
  if print_details:
    test_probs = model.predict_proba(x_test.vecs)
  evaluate(test_preds, y_test, print_details)

In [205]:
lr = LogisticRegression(C=.001, solver='newton-cg', penalty='l2')

In [206]:
lr.fit(x_tfidf, y_train)

LogisticRegression(C=0.001, solver='newton-cg')

In [207]:
training(lr, x_train_array, len(y_train), print_details=False)

Avg. F1 score: 0.6196963394342762


In [208]:
testing(lr, x_test, len(y_test))

Precision: [0.         0.62348375]
Recall: [0. 1.]
F1 score: [0.         0.76808129]
Avg. F1 score: 0.6234837457544881


C:\Users\brian\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [211]:
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='f1',error_score=0)
grid_result = grid_search.fit(x_tfidf, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.765201 using {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
0.658419 (0.011247) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.658967 (0.010571) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.658328 (0.011260) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.680628 (0.011310) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.680627 (0.011240) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.680680 (0.011253) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.735279 (0.007135) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.735237 (0.007217) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.735212 (0.007162) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.765095 (0.000394) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.765095 (0.000394) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.765095 (0.000394) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.765201 (0.000260) wi